In [2]:
!pip install transformers sentencepiece langdetect ipywidgets --quiet

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

model_name = "facebook/m2m100_418M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
supported_langs = tokenizer.lang_code_to_id.keys()

full_language_names = {
    "en": "English", "fr": "French", "de": "German", "es": "Spanish", "hi": "Hindi", "zh": "Chinese",
    "ru": "Russian", "ja": "Japanese", "ko": "Korean", "ar": "Arabic", "pt": "Portuguese", "tr": "Turkish",
    "it": "Italian", "pl": "Polish", "nl": "Dutch", "sv": "Swedish", "uk": "Ukrainian", "fi": "Finnish",
    "cs": "Czech", "el": "Greek"
}

dropdown_options = [
    (f"{full_language_names.get(code, code.title())} ({code})", code)
    for code in sorted(full_language_names.keys())
]

def translate_text(input_text, source_lang, target_lang):
    if source_lang not in supported_langs or target_lang not in supported_langs:
        return ("❌ Unsupported language.", None)
    tokenizer.src_lang = source_lang
    encoded = tokenizer(input_text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(target_lang)
    )
    translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return (f"{source_lang} → {target_lang}", translated)

display(HTML('''
<style>
    ::-webkit-scrollbar { display: none; }
    .output_wrapper, .output, .output_area { overflow: visible !important; }

    .widget-label {
        color: #80ffdb !important;
        font-weight: bold;
        font-size: 16px;
    }

    textarea, select, button {
        border-radius: 8px !important;
        border: none !important;
        font-size: 16px !important;
    }

    .widget-box {
        background: rgba(255, 255, 255, 0.06);
        backdrop-filter: blur(20px);
        -webkit-backdrop-filter: blur(20px);
        border: 1px solid rgba(255, 255, 255, 0.1);
        box-shadow: 0 8px 32px 0 rgba(31, 38, 135, 0.37);
        border-radius: 24px;
        padding: 40px;
        max-width: 550px;
        min-width: 450px;
        margin: 60px auto;
    }

    button {
        background: linear-gradient(to right, #ff6ec4, #7873f5) !important;
        color: white !important;
        font-weight: bold;
        transition: 0.3s ease-in-out;
    }

    button:hover {
        transform: scale(1.03);
        box-shadow: 0px 0px 12px #ff6ec4;
    }

    select, textarea {
        background-color: #1a1a1a !important;
        color: #ffffff !important;
    }

    .output_subarea {
        font-size: 17px;
        color: #e0f7fa;
    }

    h2 {
        font-size: 28px;
        text-align: center;
        color: #80ffdb;
        margin-bottom: 24px;
    }

    h4 {
        font-size: 20px;
        color: #4facfe;
        margin-top: 20px;
    }

    blockquote {
        font-size: 18px;
        color: #ffffff;
    }

    .centered-colab-container {
        display: flex;
        justify-content: center;
        align-items: center;
        flex-direction: column;
    }
</style>
'''))

title = widgets.HTML("<h2>🚀 AI Multi-Language Translator</h2>")

text_input = widgets.Textarea(
    placeholder='Enter your text here...',
    layout=widgets.Layout(width='100%', height='100px')
)

source_dropdown = widgets.Dropdown(
    options=dropdown_options,
    value='en',
    description='Source Language:',
    layout=widgets.Layout(width='100%')
)

target_dropdown = widgets.Dropdown(
    options=dropdown_options,
    value='hi',
    description='Target Language:',
    layout=widgets.Layout(width='100%')
)

translate_button = widgets.Button(
    description="Translate",
    layout=widgets.Layout(width='50%', height='40px')
)

output_area = widgets.Output()

main_container = widgets.VBox(
    [title, text_input, source_dropdown, target_dropdown, translate_button, output_area],
    layout=widgets.Layout(
        width='auto',
        align_items='center',
        justify_content='center',
        margin='auto'
    )
)

main_container.add_class("widget-box")

def on_translate_click(b):
    with output_area:
        output_area.clear_output()
        text = text_input.value.strip()
        source = source_dropdown.value
        target = target_dropdown.value
        if not text:
            print("⚠️ Please enter text to translate.")
            return
        original_desc = translate_button.description
        translate_button.description = "⏳ Translating..."
        translate_button.disabled = True
        try:
            lang_pair, translated = translate_text(text, source, target)
            if translated:
                display(Markdown(f"<b style='color:#00e676;'>✅ Translating:</b> `{lang_pair}`"))
                display(Markdown(f"<h4>🌍 Translated Text:</h4><blockquote>{translated}</blockquote>"))
            else:
                print(lang_pair)
        finally:
            translate_button.description = original_desc
            translate_button.disabled = False

translate_button.on_click(on_translate_click)

display(HTML('<div class="centered-colab-container">'))
display(main_container)
display(HTML('</div>'))